In [1]:
import sys
sys.path.append('/root/autodl-tmp/CommitFit')
import pandas as pd
from sklearn.model_selection import train_test_split
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.combined_model as cm 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve

2026-01-07 21:07:47.417428: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-07 21:07:47.497701: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-07 21:07:48.747168: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/root/miniconda3/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8:

In [2]:
# =========================
# 0) 配置
# =========================
CSV_PATH = "/root/autodl-tmp/CommitFit/dataset/generated_commits.csv"

# =========================
# 2) 读 CSV -> Dataset -> split
# =========================
df = pd.read_csv(CSV_PATH)
label2id={'Adaptive':0, 'Corrective':1, 'Perfective':2}
df = df.replace({"labels": label2id})
df


,user,repo,commit,labels,target_text,diffs,feature,diff_compact,source_text,pred
0,apache,hadoop,06d24042b64d6fa0e179b5845990068f849d9ce5,1,YARN-1185. Fixed FileSystemRMStateStore to not...,diff --git a/hadoop-yarn-project/CHANGES.txt b...,"[0, 0, 0, 0, 0, 0, 0, 5, 156, 86, 5, 44, 0, 0,...",diff --git a/hadoop-yarn-project/CHANGES.txt b...,Please write a concise commit message that sum...,YARN-1185. Fixed FileSystemRMStateStore to not...
1,apache,hbase,b246e68435e8d86a2a12fb8fe6c5aa9d9896ff92,2,HBASE-5259 Normalize the RegionLocation in- Ta...,diff --git a/src/main/java/org/apache/hadoop/h...,"[1, 30, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/src/main/java/org/apache/hadoop/h...,Please write a concise commit message that sum...,HBASE-10883 reverseDNS can't handle- DNS error...
2,GNOME,valadoc,ca2ba9131b10c5927172505f89f79cc0088dca3f,0,libvaladoc: gir-reader: improve short descs\n,diff --git a/src/libvaladoc/content/blockconte...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/src/libvaladoc/content/blockconte...,Please write a concise commit message that sum...,libvaladoc: Add support for empty content\n
3,arquillian,arquillian-graphene,2ce3373d1db087a001b226bccfba43285246a1d4,0,"chrome support added, when the browser is *chr...",diff --git a/api/src/main/java/org/jboss/arqui...,"[6, 28, 13, 1, 20, 27, 1, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/api/src/main/java/org/jboss/arqui...,Please write a concise commit message that sum...,ARQGRA-470: Selenium's network traffic capture...
4,apache,hadoop,a3ecc3b910aa3bbc3ede2b8ba1bd040d02a26ca8,0,HDFS-4733. Make HttpFS username pattern- confi...,diff --git a/hadoop-hdfs-project/hadoop-hdfs-h...,"[3, 12, 0, 6, 0, 0, 0, 2, 76, 4, 3, 17, 0, 0, ...",diff --git a/hadoop-hdfs-project/hadoop-hdfs-h...,Please write a concise commit message that sum...,HDFS-4347 Add support for user groups in web a...
...,...,...,...,...,...,...,...,...,...,...
263,kiegroup,drools,8b9ffeb5f2883e3d5187ad2f4a3a35540ff70326,1,JBRULES-527: fixing compilation problems--git-...,diff --git a/drools-core/src/main/java/org/dro...,"[3, 28, 6, 15, 0, 50, 2, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-core/src/main/java/org/dro...,Please write a concise commit message that sum...,JBRULES-3135: Fixed bug in reteoo--git-svn-id:...
264,apache,hadoop,ed77c8925d7126f9ea3c8d9cbb1e246ad61ce37c,1,YARN-596. Use scheduling policies throughout t...,diff --git a/hadoop-yarn-project/CHANGES.txt b...,"[11, 131, 38, 16, 28, 2, 8, 3, 35, 29, 42, 6, ...",diff --git a/hadoop-yarn-project/CHANGES.txt b...,Please write a concise commit message that sum...,YARN-596. Use scheduling policies throughout t...
265,apache,deltaspike,4600b51887de35647220ca47fa1a09042db2cb52,1,DELTASPIKE-286 Comparators should be Serializa...,diff --git a/deltaspike/core/api/src/main/java...,"[1, 6, 7, 10, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/deltaspike/core/api/src/main/java...,Please write a concise commit message that sum...,DELTASPIKE-289 add statics order\n
266,kiegroup,drools,d51eaf7a642b4b4572aa9628e66b760e5c3ee2d5,1,JBRULES-1102 Bug in DefaultBetaConstraint clas...,diff --git a/drools-core/src/main/java/org/dro...,"[5, 40, 2, 4, 1, 0, 0, 1, 246, 0, 0, 3, 0, 0, ...",diff --git a/drools-core/src/main/java/org/dro...,Please write a concise commit message that sum...,JBRULES-303: fixing indexing of fields--git-sv...


In [3]:
test = df
test

,user,repo,commit,labels,target_text,diffs,feature,diff_compact,source_text,pred
0,apache,hadoop,06d24042b64d6fa0e179b5845990068f849d9ce5,1,YARN-1185. Fixed FileSystemRMStateStore to not...,diff --git a/hadoop-yarn-project/CHANGES.txt b...,"[0, 0, 0, 0, 0, 0, 0, 5, 156, 86, 5, 44, 0, 0,...",diff --git a/hadoop-yarn-project/CHANGES.txt b...,Please write a concise commit message that sum...,YARN-1185. Fixed FileSystemRMStateStore to not...
1,apache,hbase,b246e68435e8d86a2a12fb8fe6c5aa9d9896ff92,2,HBASE-5259 Normalize the RegionLocation in- Ta...,diff --git a/src/main/java/org/apache/hadoop/h...,"[1, 30, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/src/main/java/org/apache/hadoop/h...,Please write a concise commit message that sum...,HBASE-10883 reverseDNS can't handle- DNS error...
2,GNOME,valadoc,ca2ba9131b10c5927172505f89f79cc0088dca3f,0,libvaladoc: gir-reader: improve short descs\n,diff --git a/src/libvaladoc/content/blockconte...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/src/libvaladoc/content/blockconte...,Please write a concise commit message that sum...,libvaladoc: Add support for empty content\n
3,arquillian,arquillian-graphene,2ce3373d1db087a001b226bccfba43285246a1d4,0,"chrome support added, when the browser is *chr...",diff --git a/api/src/main/java/org/jboss/arqui...,"[6, 28, 13, 1, 20, 27, 1, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/api/src/main/java/org/jboss/arqui...,Please write a concise commit message that sum...,ARQGRA-470: Selenium's network traffic capture...
4,apache,hadoop,a3ecc3b910aa3bbc3ede2b8ba1bd040d02a26ca8,0,HDFS-4733. Make HttpFS username pattern- confi...,diff --git a/hadoop-hdfs-project/hadoop-hdfs-h...,"[3, 12, 0, 6, 0, 0, 0, 2, 76, 4, 3, 17, 0, 0, ...",diff --git a/hadoop-hdfs-project/hadoop-hdfs-h...,Please write a concise commit message that sum...,HDFS-4347 Add support for user groups in web a...
...,...,...,...,...,...,...,...,...,...,...
263,kiegroup,drools,8b9ffeb5f2883e3d5187ad2f4a3a35540ff70326,1,JBRULES-527: fixing compilation problems--git-...,diff --git a/drools-core/src/main/java/org/dro...,"[3, 28, 6, 15, 0, 50, 2, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-core/src/main/java/org/dro...,Please write a concise commit message that sum...,JBRULES-3135: Fixed bug in reteoo--git-svn-id:...
264,apache,hadoop,ed77c8925d7126f9ea3c8d9cbb1e246ad61ce37c,1,YARN-596. Use scheduling policies throughout t...,diff --git a/hadoop-yarn-project/CHANGES.txt b...,"[11, 131, 38, 16, 28, 2, 8, 3, 35, 29, 42, 6, ...",diff --git a/hadoop-yarn-project/CHANGES.txt b...,Please write a concise commit message that sum...,YARN-596. Use scheduling policies throughout t...
265,apache,deltaspike,4600b51887de35647220ca47fa1a09042db2cb52,1,DELTASPIKE-286 Comparators should be Serializa...,diff --git a/deltaspike/core/api/src/main/java...,"[1, 6, 7, 10, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/deltaspike/core/api/src/main/java...,Please write a concise commit message that sum...,DELTASPIKE-289 add statics order\n
266,kiegroup,drools,d51eaf7a642b4b4572aa9628e66b760e5c3ee2d5,1,JBRULES-1102 Bug in DefaultBetaConstraint clas...,diff --git a/drools-core/src/main/java/org/dro...,"[5, 40, 2, 4, 1, 0, 0, 1, 246, 0, 0, 3, 0, 0, ...",diff --git a/drools-core/src/main/java/org/dro...,Please write a concise commit message that sum...,JBRULES-303: fixing indexing of fields--git-sv...


In [4]:
# train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=10, random_state=42))

In [5]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('/root/autodl-tmp/models/google-bert/bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/models/google-bert/bert-base-uncased')

codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/models/codebert-base')

In [6]:
import torch

# Create Datasets and DataLoaders
# train_dataset = preprocesser.SentencePairDataset(train, bert_tokenizer, codebert_tokenizer,message='msgs',command='diff_compact',label='labels')
# val_dataset = preprocesser.SentencePairDataset(val, bert_tokenizer, codebert_tokenizer,message='msgs',command='diff_compact',label='labels')
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='pred',command='diff_compact',label='labels')
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



from transformers import AutoModelForSequenceClassification
# 自动选择设备（GPU 优先）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('/root/autodl-tmp/models/google-bert/bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/models/google-bert/bert-base-uncased')

codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/models/codebert-base')

# 加载
ckpt = torch.load("/root/autodl-tmp/CommitFit/notebooks/E-3-best(70%)/reward_model_combined.pt")
# 重新构建结构（必须和训练时一致）
base_model1 = cm.BaseModel(bert_model)
base_model2 = cm.BaseModel(codebert_model)
reward_model = cm.CombinedModel(
    base_model1=base_model1,
    base_model2=base_model2
    # hidden_dim=???  如果你的 __init__ 需要，就得你自己传训练时的值
).to(device)

reward_model.load_state_dict(ckpt["model_state_dict"])
reward_model.eval()

CombinedModel(
  (base_model1): BaseModel(
    (transformer_model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=76

In [7]:
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='pred',command='source_text',label='labels')
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = reward_model.evaluate(test_loader)

evaluate: 100%|██████████| 9/9 [00:04<00:00,  1.97batch/s]

Validation Accuracy: 0.5746268656716418
Precision: 0.5795160857133256
Recall: 0.5746268656716418
F1-Score: 0.5746860923419873


In [9]:
from sklearn import metrics

In [10]:
cm = metrics.confusion_matrix(test_labels, test_predictions)
cm

array([[48, 19, 12],
       [17, 53, 19],
       [22, 25, 53]])

In [11]:
# import torch
# # 保存
# torch.save(
#     {
#         "model_state_dict": combined_model.state_dict(),
#         "num_labels": 3,
#     },
#     "./reward_model_combined.pt",
# )

# # 加载
# ckpt = torch.load("reward_model_combined.pt")
# combined_model.load_state_dict(ckpt["model_state_dict"])

In [12]:
# 加载
# reward_model = torch.load("reward_model_combined.pth")
# combined_model.load_state_dict(ckpt["model_state_dict"])

In [13]:
#发送多种类型的邮件
from email.mime.multipart import MIMEMultipart
import smtplib

from email.mime.text import MIMEText
msg_from = '915803745@qq.com'  # 发送方邮箱
passwd = 'vcuosuurrgkfbdai'   #就是上面的授权码
 
# to= ['g.zhang@gotion.com', 'j.tong@gotion.com'] #接受方邮箱
to= ['j.tong@gotion.com'] #接受方邮箱
#设置邮件内容
#MIMEMultipart类可以放任何内容
msg = MIMEMultipart()
conntent=f"强化学习"
#把内容加进去
msg.attach(MIMEText(conntent,'plain','utf-8'))
 
#设置邮件主题
msg['Subject']="强化学习模型训练完毕"
 
#发送方信息
msg['From']=msg_from
 
#开始发送
 
#通过SSL方式发送，服务器地址和端口
s = smtplib.SMTP_SSL("smtp.qq.com", 465)
# 登录邮箱
s.login(msg_from, passwd)
#开始发送
s.sendmail(msg_from,to,msg.as_string())
print("强化学习模型训练完毕")

强化学习模型训练完毕
